In [1]:
import pandas as pd 
import numpy as np 
from collections import namedtuple
from geocodio import GeocodioClient
import os 

In [2]:
data=pd.read_csv('FoodInspections-morecleaning.csv')
data.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328.0,Restaurant,Risk 1 (High),2121 S Prairie Ave,Chicago,IL,60616.0,2017-08-28T00:00:00Z,License,Pass,NaN,41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
1,2079129,JET'S PIZZA,JET'S PIZZA,2522268.0,Restaurant,Risk 2 (Medium),1025 W Madison St,Chicago,IL,60607.0,2017-08-28T00:00:00Z,License,Not Ready,NaN,41.881572,-87.653052,"(41.88157249576794, -87.65305233593274)"
2,2079125,ROOM 1520,ROOM 1520,2446638.0,Special Event,Risk 3 (Low),301 N Justine St,Chicago,IL,60607.0,2017-08-28T00:00:00Z,License Re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328,"(41.88657652150854, -87.66532812140231)"
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338.0,Restaurant,Risk 1 (High),2121 S Prairie Ave,Chicago,IL,60616.0,2017-08-28T00:00:00Z,License,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079.0,School Cafeteria,Risk 1 (High),2245 N Mcvicker Ave,Chicago,IL,60639.0,2017-08-28T00:00:00Z,License Re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711,"(41.921675488910864, -87.7767113569357)"


In [4]:
API_KEY = os.environ['GEOCODIO_API_KEY']
client = GeocodioClient(API_KEY, timeout=15)

In [5]:
print(len(data[data['Location'].isnull()])) 

544


In [15]:
for index, row in data.iterrows():
    if row['Location'] is np.nan or row['Location'] is None:
        address = row['Address']
        if address is np.nan: continue 
        address = address.replace('(','').replace(')','')
        city =  str(row['City']).replace('nan','Chicago') or 'Chicago'
        state = row['State'].replace('nan', 'IL') or 'IL'
        zipcode  = str(row['Zip'])[:5].replace('nan','') or str(60613)
        full_address = f'{address}, {city}, {state} {zipcode}'.replace('nan','')
        full_address = full_address.replace('(','').replace(')','')
        print(full_address)
        components = {"formatted_street":address, "city":city, "state": state}
        if zipcode: components['zip'] = zipcode 
        
        print(components )
        location = client.geocode_address(full_address, limit=1)
        print(f"Location Returned: {location}")
        if location is not None:
            lat = location.coords[0]
            lon = location.coords[1]
            data.at[index, 'Location'] = str(location.coords )
            data.at[index, 'Latitude'] = lat 
            data.at[index, 'Longitude'] = lon
            try:
                match = location['results'][0]
                address_details =  match['address_components']
            except Exception as e:
                print(f'Could not get address for {full_address} details due to {str(e)}')
            if row['City'] is np.nan:
                data.at[index,'City'] = address_details.get('city')
            if row['State'] is np.nan:
                data.at[index,'State'] = address_details.get('state')
            if row['Zip'] is np.nan:
                data.at[index,'Zip'] = float(address_details.get('zip')) 
    if row['City'] is np.nan and row['Location'] is not np.nan: 
        coords = row['Location'].replace('(','').replace(')','').split(',')
        lat = float(coords[0].strip())
        lon = float(coords[1].strip())  
        lookup_cord = tuple([lat, lon])

        print(f'Looking up city for {lookup_cord}')
        location = client.reverse(lookup_cord, limit=1)
        try: 
            match = location['results'][0]
            address_details =  match['address_components']
        except Exception as e:
            print(f'Could not get City for {coords} details due to {str(e)}')
        else:
            data.at[index,'City'] = address_details.get('city')
    if row['State'] is np.nan and row['Location'] is not np.nan:
        coords = row['Location'].replace('(','').replace(')','').split(',')
        lat = float(coords[0].strip())
        lon = float(coords[1].strip())
        lookup_cord = tuple([lat, lon])
        print(f'Looking up state for {lookup_cord}')
        location = client.reverse(lookup_cord, limit=1)
        try: 
            match = location['results'][0]
            address_details =  match['address_components']
        except Exception as e:
            print(f'Could not get State for {coords} details due to {str(e)}')
        else:
            data.at[index,'State'] = address_details.get('state')
    if row['Zip'] is np.nan and row['Location'] is not np.nan: 
        coords = row['Location'].replace('(','').replace(')','').split(',')
        lat = float(coords[0].strip())
        lon = float(coords[1].strip())     
        lookup_cord = tuple([lat, lon])

        print(f'Looking up zip for {lookup_cord}')
        location = client.reverse(lookup_cord, limit=1)
        try: 
            match = location['results'][0]
            address_details =  match['address_components']
        except Exception as e:
            print(f'Could not get Zip for {coords} details due to {str(e)}')
        else:
            data.at[index,'Zip'] = float(address_details.get('zip')) 


{'formatted_street': '1509 Sibley Blvd', 'city': 'Calumet City', 'state': 'IL', 'zip': '60409'}
Location Returned: {'input': {'address_components': {'number': '1509', 'street': 'Sibley', 'suffix': 'Blvd', 'formatted_street': 'Sibley Blvd', 'city': 'Calumet City', 'state': 'IL', 'zip': '60409', 'country': 'US'}, 'formatted_address': '1509 Sibley Blvd, Calumet City, IL 60409'}, 'results': [{'address_components': {'number': '1509', 'street': 'Sibley', 'suffix': 'Blvd', 'formatted_street': 'Sibley Blvd', 'city': 'Calumet City', 'county': 'Cook County', 'state': 'IL', 'zip': '60409', 'country': 'US'}, 'formatted_address': '1509 Sibley Blvd, Calumet City, IL 60409', 'location': {'lat': 41.622431, 'lng': -87.561956}, 'accuracy': 1, 'accuracy_type': 'rooftop', 'source': 'Cook'}]}
1509 Sibley Blvd, Calumet City, IL 60409
{'formatted_street': '1509 Sibley Blvd', 'city': 'Calumet City', 'state': 'IL', 'zip': '60409'}
Location Returned: {'input': {'address_components': {'number': '1509', 'street':

In [37]:
for index, row in data.iterrows():
    if (row['Zip'] is np.nan or str(row['Zip']) == 'nan') and row['Location'] is not np.nan: 
        coords = row['Location'].replace('(','').replace(')','').split(',')
        lat = float(coords[0].strip())
        lon = float(coords[1].strip())     
        lookup_cord = tuple([lat, lon])

        print(f'Looking up zip for {lookup_cord}')
        location = client.reverse(lookup_cord, limit=1)
        print(f'Location Returned {location}')
        try: 
            match = location['results'][0]
            address_details =  match['address_components']
        except Exception as e:
            print(f'Could not get Zip for {coords} details due to {str(e)}')
        else:
            new_zip = float(address_details.get('zip'))
            print(f'setting zip to {new_zip}')
            data.at[index,'Zip'] = new_zip  


Looking up zip for (41.707888, -87.807084)
Location Returned {'results': [{'address_components': {'number': '7451', 'predirectional': 'W', 'street': '100th', 'suffix': 'Pl', 'formatted_street': 'W 100th Pl', 'city': 'Bridgeview', 'county': 'Cook County', 'state': 'IL', 'zip': '60455', 'country': 'US'}, 'formatted_address': '7451 W 100th Pl, Bridgeview, IL 60455', 'location': {'lat': 41.707888, 'lng': -87.807084}, 'accuracy': 1, 'accuracy_type': 'rooftop', 'source': 'Cook'}]}
setting zip to 60455.0
Looking up zip for (41.88406738126699, -87.63373557888185)
Location Returned {'results': [{'address_components': {'number': '181', 'predirectional': 'W', 'street': 'Randolph', 'suffix': 'St', 'formatted_street': 'W Randolph St', 'city': 'Chicago', 'county': 'Cook County', 'state': 'IL', 'zip': '60601', 'country': 'US'}, 'formatted_address': '181 W Randolph St, Chicago, IL 60601', 'location': {'lat': 41.884145, 'lng': -87.63364}, 'accuracy': 1, 'accuracy_type': 'rooftop', 'source': 'Cook'}, {'

In [47]:
print( data[data['Address'].isnull()])

        Inspection ID                    DBA Name                    AKA Name  \
28439         1763245  EAT N RUN CHICKEN AND FISH  EAT N RUN CHICKEN AND FISH   
148160         114452              STARFRUIT CAFE              STARFRUIT CAFE   
149674          60405      CASA CENTRAL LA POSADA      CASA CENTRAL LA POSADA   

        License #  Facility Type             Risk Address     City State  \
28439   2442868.0     Restaurant    Risk 1 (High)     NaN      NaN    IL   
148160  2031651.0  Other/unknown  Risk 2 (Medium)     NaN  Chicago    IL   
149674        0.0     Restaurant    Risk 1 (High)     NaN      NaN    IL   

            Zip       Inspection Date Inspection Type    Results  \
28439       NaN  2016-04-19T00:00:00Z         License  Not Ready   
148160  60654.0  2010-05-05T00:00:00Z         License       Fail   
149674      NaN  2010-04-05T00:00:00Z         Canvass       Pass   

                                               Violations  Latitude  \
28439                     

In [38]:
print('Number of null values in each column: ')
print(data.isnull().sum())

Number of null values in each column: 
Inspection ID          0
DBA Name               0
AKA Name            2543
License #             15
Facility Type          0
Risk                  66
Address                3
City                   2
State                  0
Zip                    2
Inspection Date        0
Inspection Type        1
Results                0
Violations         30798
Latitude               3
Longitude              3
Location               3
dtype: int64


In [39]:
data.to_csv('FoodInspection_location.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c00fda9-e408-45c6-ac1e-09c6fb54e7f2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>